In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import reshape
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dense, Dropout
from keras.utils import to_categorical

2024-11-03 17:43:40.151557: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Import Data 

In [2]:
path = R'/Users/shaquillelee/Documents/Specialization /Datasets'

In [3]:
weather_df = pd.read_csv(os.path.join(path, 'Prepared Data', 'weather_cleaned.csv'), index_col = 0)
pleasant_df = pd.read_csv(os.path.join(path, 'Prepared Data', 'pleasant_cleaned.csv'), index_col = 0)

In [4]:
weather_df.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [5]:
weather_df.shape

(22950, 135)

In [6]:
pleasant_df.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
pleasant_df.shape

(22950, 15)

# Split Data 

In [8]:
# Create 'X' Matrix 
X = weather_df
y = pleasant_df

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [11]:
X = X.reshape(-1,15,9)

In [12]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [13]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360, 15)
(4590, 15, 9) (4590, 15)


# 3. Creating CNN (Convolution Neural Network) model

In [14]:
stations = {
    0: 'BASEL',
    1: 'BELGRADE',
    2: 'BUDAPEST',
    3: 'DEBILT',
    4: 'DUSSELDORF',
    5: 'HEATHROW',
    6: 'KASSEL',
    7: 'LJUBLJANA',
    8: 'MAASTRICHT',
    9: 'MADRID',
   10: 'MUNCHENB',
   11: 'OSLO',
   12: 'SONNBLICK',
   13: 'STOCKHOLM',
   14: 'VALENTIA',
}

In [15]:
# Create a Keras leyered model. Use initial hyperparameters: 4, 16, 18, softmax

epochs = 4
batch_size = 16
n_hidden = 18

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #sigmoid

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)


Epoch 1/4
1148/1148 - 2s - 2ms/step - accuracy: 0.1338 - loss: 2905.3079
Epoch 2/4
1148/1148 - 1s - 1ms/step - accuracy: 0.1327 - loss: 29161.4727
Epoch 3/4
1148/1148 - 1s - 1ms/step - accuracy: 0.1286 - loss: 99541.4531
Epoch 4/4
1148/1148 - 1s - 1ms/step - accuracy: 0.1276 - loss: 217126.2344


In [18]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [19]:
#Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BELGRADE  BUDAPEST  DEBILT  HEATHROW  KASSEL  LJUBLJANA  \
True                                                                  
BASEL              2      1748      76       130     228         45   
BELGRADE           0       793       0         0       6          0   
BUDAPEST           0       133       0         0       0          0   
DEBILT             0        43       0         0       0          0   
DUSSELDORF         0        23       0         0       0          0   
HEATHROW           0        65       0         0       0          0   
KASSEL             0         6       0         0       0          0   
LJUBLJANA          0        41       0         0       0          0   
MAASTRICHT         0         4       0         0       1          0   
MADRID             0       213       0         3       3          2   
MUNCHENB           0         6       0         0       0          0   
OSLO               0         1    

In [20]:
# Create a Keras leyered model. Use initial hyperparameters: 16, 18, 18, softmax

epochs = 16
batch_size = 18
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #softmax

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/16
1020/1020 - 2s - 2ms/step - accuracy: 0.1505 - loss: 2735.6116
Epoch 2/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1474 - loss: 28712.2832
Epoch 3/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1356 - loss: 93502.1094
Epoch 4/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1307 - loss: 206251.5000
Epoch 5/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1246 - loss: 372521.1562
Epoch 6/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1242 - loss: 602205.2500
Epoch 7/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1221 - loss: 881861.5000
Epoch 8/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1213 - loss: 1231991.2500
Epoch 9/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1203 - loss: 1675809.8750
Epoch 10/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1192 - loss: 2185532.2500
Epoch 11/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1168 - loss: 2789798.2500
Epoch 12/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1187 - loss: 3488867.0000
Epoch 13/16
1020/1020 - 1s - 1ms/step - accuracy: 0.1189 - loss: 4256197.0000
Epoch 

In [23]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [24]:
#Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         180       165       742     187         710         1      79   
BELGRADE        1        38       479      24         296         0       7   
BUDAPEST        0         0        70       8          76         0       3   
DEBILT          0         0        23       5          23         0       5   
DUSSELDORF      0         0         8       2          13         0       1   
HEATHROW        1         0        18       8          40         0       5   
KASSEL          0         0         0       1           6         0       0   
LJUBLJANA       0         0        17       1          13         0       4   
MAASTRICHT      0         0         3       0           3         0       0   
MADRID         20        16        92       9          92         0      12   
MUNCHENB 

In [25]:
# Create a Keras leyered model. Use initial hyperparameters: 16, 18, 18, softtmax

epochs = 30 
batch_size = 32
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #softmax

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
574/574 - 1s - 2ms/step - accuracy: 0.0830 - loss: 1833.6627
Epoch 2/30
574/574 - 1s - 987us/step - accuracy: 0.0997 - loss: 19545.0859
Epoch 3/30
574/574 - 1s - 1ms/step - accuracy: 0.1054 - loss: 69490.7734
Epoch 4/30
574/574 - 1s - 1ms/step - accuracy: 0.1133 - loss: 156529.2500
Epoch 5/30
574/574 - 1s - 999us/step - accuracy: 0.1189 - loss: 282540.3438
Epoch 6/30
574/574 - 1s - 985us/step - accuracy: 0.1197 - loss: 457073.1250
Epoch 7/30
574/574 - 1s - 1ms/step - accuracy: 0.1197 - loss: 678557.0000
Epoch 8/30
574/574 - 1s - 990us/step - accuracy: 0.1218 - loss: 950489.6250
Epoch 9/30
574/574 - 1s - 987us/step - accuracy: 0.1194 - loss: 1255936.5000
Epoch 10/30
574/574 - 1s - 996us/step - accuracy: 0.1178 - loss: 1622980.2500
Epoch 11/30
574/574 - 1s - 984us/step - accuracy: 0.1196 - loss: 2052551.3750
Epoch 12/30
574/574 - 1s - 995us/step - accuracy: 0.1198 - loss: 2530276.2500
Epoch 13/30
574/574 - 1s - 988us/step - accuracy: 0.1198 - loss: 3037934.7500
Epoch 14/30
574

In [28]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [29]:
#Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL          73       624        54      14         118       465      71   
BELGRADE        0       392         0       0          30       104       4   
BUDAPEST        0        58         0       0           8        25       1   
DEBILT          0        22         0       0           6        20       2   
DUSSELDORF      0         6         0       0           2        11       0   
HEATHROW        0         9         0       0           0        27       1   
KASSEL          0         0         0       0           1         1       0   
LJUBLJANA       0         5         0       0           0         7       0   
MAASTRICHT      0         2         0       0           1         1       0   
MADRID          0        38         1       0           7        92       8   
MUNCHENB   

In [40]:
# Create a Keras leyered model. Use initial hyperparameters: 30, 32, 124, softtmax

epochs = 30 
batch_size = 32
n_hidden = 124

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #softmax

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
574/574 - 2s - 3ms/step - accuracy: 0.1122 - loss: 3322.4768
Epoch 2/30
574/574 - 1s - 1ms/step - accuracy: 0.1092 - loss: 28904.5176
Epoch 3/30
574/574 - 1s - 1ms/step - accuracy: 0.1135 - loss: 98123.7734
Epoch 4/30
574/574 - 1s - 1ms/step - accuracy: 0.1121 - loss: 222794.7344
Epoch 5/30
574/574 - 1s - 1ms/step - accuracy: 0.1141 - loss: 410069.5938
Epoch 6/30
574/574 - 1s - 1ms/step - accuracy: 0.1163 - loss: 655449.1250
Epoch 7/30
574/574 - 1s - 1ms/step - accuracy: 0.1168 - loss: 940525.8125
Epoch 8/30
574/574 - 1s - 1ms/step - accuracy: 0.1156 - loss: 1340234.1250
Epoch 9/30
574/574 - 1s - 1ms/step - accuracy: 0.1181 - loss: 1748423.5000
Epoch 10/30
574/574 - 1s - 1ms/step - accuracy: 0.1207 - loss: 2288331.7500
Epoch 11/30
574/574 - 1s - 1ms/step - accuracy: 0.1216 - loss: 2831586.5000
Epoch 12/30
574/574 - 1s - 1ms/step - accuracy: 0.1241 - loss: 3477688.0000
Epoch 13/30
574/574 - 1s - 1ms/step - accuracy: 0.1247 - loss: 4270880.5000
Epoch 14/30
574/574 - 1s - 1ms/s

In [43]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [44]:
#Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL           2       570       175      76         291        17     142   
BELGRADE        0       348         0       5          78         0      38   
BUDAPEST        0        32         0       3          18         0      21   
DEBILT          0        14         0       1           7         0       6   
DUSSELDORF      0         2         0       0           6         0       1   
HEATHROW        0         3         0       0          15         0      13   
KASSEL          0         0         0       0           0         0       2   
LJUBLJANA       0         6         0       0           3         0       4   
MAASTRICHT      0         2         0       0           1         0       0   
MADRID          0        16         0      23          35         2      28   
MUNCHENB   

In [30]:
# Create a Keras leyered model. Use initial hyperparameters: 30, 32, 64, sigmoid

epochs = 30 
batch_size = 32
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) #sigmoid

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
574/574 - 1s - 2ms/step - accuracy: 0.1200 - loss: 2645.5669
Epoch 2/30
574/574 - 1s - 1ms/step - accuracy: 0.1242 - loss: 21707.7676
Epoch 3/30
574/574 - 1s - 1ms/step - accuracy: 0.1285 - loss: 75708.0625
Epoch 4/30
574/574 - 1s - 1000us/step - accuracy: 0.1275 - loss: 170893.7031
Epoch 5/30
574/574 - 1s - 998us/step - accuracy: 0.1295 - loss: 305412.1875
Epoch 6/30
574/574 - 1s - 998us/step - accuracy: 0.1229 - loss: 497042.5625
Epoch 7/30
574/574 - 1s - 999us/step - accuracy: 0.1191 - loss: 711435.9375
Epoch 8/30
574/574 - 1s - 994us/step - accuracy: 0.1156 - loss: 992860.5625
Epoch 9/30
574/574 - 1s - 1ms/step - accuracy: 0.1169 - loss: 1265921.0000
Epoch 10/30
574/574 - 1s - 997us/step - accuracy: 0.1188 - loss: 1586082.5000
Epoch 11/30
574/574 - 1s - 997us/step - accuracy: 0.1198 - loss: 1938110.5000
Epoch 12/30
574/574 - 1s - 994us/step - accuracy: 0.1185 - loss: 2345224.2500
Epoch 13/30
574/574 - 1s - 996us/step - accuracy: 0.1190 - loss: 2749960.5000
Epoch 14/30
57

In [33]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [34]:
#Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL
True             
BASEL        2932
BELGRADE      901
BUDAPEST      167
DEBILT         61
DUSSELDORF     26
HEATHROW       80
KASSEL          7
LJUBLJANA      44
MAASTRICHT      7
MADRID        350
MUNCHENB        6
OSLO            4
STOCKHOLM       2
VALENTIA        3


In [35]:
# Create a Keras leyered model. Use initial hyperparameters: 30, 32, 64, tanh

epochs = 30 
batch_size = 32
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) #tanh

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
574/574 - 2s - 3ms/step - accuracy: 0.0116 - loss: 23.7871
Epoch 2/30
574/574 - 1s - 1ms/step - accuracy: 0.0343 - loss: 25.3027
Epoch 3/30
574/574 - 1s - 1ms/step - accuracy: 0.0452 - loss: 25.5835
Epoch 4/30
574/574 - 1s - 1ms/step - accuracy: 0.0456 - loss: 25.5835
Epoch 5/30
574/574 - 1s - 1ms/step - accuracy: 0.0462 - loss: 25.5835
Epoch 6/30
574/574 - 1s - 1ms/step - accuracy: 0.0465 - loss: 25.5835
Epoch 7/30
574/574 - 1s - 1ms/step - accuracy: 0.0462 - loss: 25.5835
Epoch 8/30
574/574 - 1s - 1ms/step - accuracy: 0.0462 - loss: 25.5835
Epoch 9/30
574/574 - 1s - 1ms/step - accuracy: 0.0466 - loss: 25.5835
Epoch 10/30
574/574 - 1s - 1ms/step - accuracy: 0.0479 - loss: 25.5502
Epoch 11/30
574/574 - 1s - 1ms/step - accuracy: 0.1713 - loss: 22.5864
Epoch 12/30
574/574 - 1s - 1ms/step - accuracy: 0.1944 - loss: 22.2396
Epoch 13/30
574/574 - 1s - 1ms/step - accuracy: 0.1957 - loss: 22.2388
Epoch 14/30
574/574 - 1s - 1ms/step - accuracy: 0.1965 - loss: 22.2388
Epoch 15/30
574

In [38]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [39]:
#Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  DUSSELDORF  KASSEL  MAASTRICHT
True                                             
BASEL         883         871     928         250
BELGRADE      471         409      10          11
BUDAPEST       62         105       0           0
DEBILT         43          18       0           0
DUSSELDORF      8          17       1           0
HEATHROW       31          46       1           2
KASSEL          4           3       0           0
LJUBLJANA      17          24       0           3
MAASTRICHT      0           5       0           2
MADRID         36         250      54          10
MUNCHENB        0           6       0           0
OSLO            1           3       0           0
STOCKHOLM       1           1       0           0
VALENTIA        0           2       0           1
